In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# %reload_ext dotenv

# %dotenv ../../env/.env

# import warnings
# import utils
# from pandas import Timedelta
# import time
# from utils import logger, sql_to_df, run_sql, session, conn
# import pandas as pd
# warnings.filterwarnings('ignore')

# try:
#     %load_ext autotime
# except:
#     !pip install ipython-autotime
#     %load_ext autotime

In [ ]:
import logging
logger.setLevel(logging.INFO)

In [ ]:
# # warehouse_ids = [41,42,43]
# warehouse_ids = [38,39,40]
# joined = ','.join([str(w) for w in warehouse_ids])
# sql_str = f"({joined})"

# sql = f"""
# select 
#     warehouse_id, 
#     era_start, 
#     era_end
# from query_era_test

# -- where warehouse_id in {sql_str}
# order by era_start asc
# """

# df = sql_to_df(sql)
# display(df)

In [ ]:
for warehouse_id in df.warehouse_id.unique():
    print(warehouse_id)
    wh_df = df[df.warehouse_id == warehouse_id].sort_values('era_start', ascending=True)

In [ ]:
pd.timedelta_range(wh_df.era_start.min(), wh_df.era_end.max(), freq="1min")


In [ ]:
from pandas import Timestamp
query_eras = \
[(Timestamp('2023-04-10 16:44:00'), Timestamp('2023-04-10 16:44:10')),
 (Timestamp('2023-04-10 16:44:15'), Timestamp('2023-04-10 16:44:25')),
 (Timestamp('2023-04-10 16:44:55'), Timestamp('2023-04-10 16:45:05')),
 (Timestamp('2023-04-10 16:45:15'), Timestamp('2023-04-10 16:45:25'))]

qdf = pd.DataFrame(query_eras, columns=['era_start', 'era_end'])
qdf

In [ ]:
# %pip install duckdb

In [ ]:
import snowflake.snowpark.functions as F
import snowflake.snowpark.dataframe

polling_sec = 1
# where warehouse_id in (
#     select warehouse_id from (
#         select top 10
#             warehouse_id,
#             warehouse_name,
#             sum(dollars_used)
#         from warehouse_metering_history_enriched
#         group by 1,2 order by 3 desc
#     )


def greedy_suspend(query_eras, autosuspend_sec):
    started = False
    resumed_on = None
    previous_end = None
    dt = Timedelta(seconds=autosuspend_sec)
    events = []
    for (start, end) in query_eras:
        # first process events between previous_end and start
        # suspend happens at: 
        # previous_end + 2*polling_period, as long as we're after resumed_on + 60 + 2*polling_period, 
        if started:
            timediff = start - previous_end 
            if timediff > dt:
                suspend_time = previous_end + dt
                started = False
                events.append((suspend_time, 'suspend'))

        if not started:
            started = True
            resumed_on = start
            events.append((resumed_on, 'resume'))

        previous_end = end
    
    if started:
        suspend_time = previous_end + dt
        started = False
        events.append((suspend_time, 'suspend')) 
        
    events_df = pd.DataFrame(events, columns=['timestamp', 'event_type'])
    return events_df

def snowflake_suspend(query_eras, autosuspend_sec):
#     dt = Timedelta(seconds=autosuspend_sec)
#     sixty = Timedelta(seconds=60)
    return greedy_suspend(query_eras, max(autosuspend_sec, 30))

def smart_suspend(query_eras, polling_sec):
    started = False
    running = 0
    resumed_on = None

    previous_end = None
    resumed_plus_sixty = None
    events = []
    for (start, end) in query_eras:
        
        # first process events between previous_end and start
        # suspend happens at: 
        # previous_end + 2*polling_period, as long as we're after resumed_on + 60 + 2*polling_period, 

        if started:
            first_check_after_previous_end = previous_end + 1*Timedelta(seconds=polling_sec)
            earliest_possible_suspend = max(first_check_after_previous_end, resumed_plus_sixty)
            if earliest_possible_suspend < start:
                started = False
                events.append((earliest_possible_suspend, 'suspend'))

        if not started:
            started = True
            resumed_on = start
            resumed_plus_sixty = resumed_on + Timedelta(seconds=60)
            events.append((resumed_on, 'resume'))

        previous_end = end
    
    if started:
        first_check_after_previous_end = previous_end + 1*Timedelta(seconds=polling_sec)
        earliest_possible_suspend = max(first_check_after_previous_end, resumed_plus_sixty)
        events.append((earliest_possible_suspend, 'suspend'))


    events_df = pd.DataFrame(events, columns=['timestamp', 'event_type'])
    return events_df

def wh_events_sim(df):
    warehouse_event_dfs = []
    for warehouse_id in df.warehouse_id.unique():
        print(warehouse_id)
        wh_df = df[df.warehouse_id == warehouse_id].sort_values('era_start', ascending=True)

        eras = list(zip(wh_df.era_start, wh_df.era_end))

        strategy = {'type': 'greedy_after_one_min', 'polling_sec': polling_sec}

        events_df = smart_suspend(eras, polling_sec)
        events_df['strategy'] = str(strategy)
        events_df['warehouse_id'] = warehouse_id
        warehouse_event_dfs.append(events_df)

        # autosuspend_sec = 60
        for autosuspend_sec in [30, 60, 120]:

            strategy = {'type': 'greedy', 'autosuspend_sec': autosuspend_sec}
            # print(strategy)
            events_df = greedy_suspend(eras, autosuspend_sec)
            events_df['strategy'] = str(strategy)
            events_df['warehouse_id'] = warehouse_id
            warehouse_event_dfs.append(events_df)

            strategy = {'type': 'snowflake', 'autosuspend_sec': autosuspend_sec}
            # print(strategy)
            events_df = snowflake_suspend(eras, autosuspend_sec)
            events_df['strategy'] = str(strategy)
            events_df['warehouse_id'] = warehouse_id
            warehouse_event_dfs.append(events_df)

    final_df = pd.concat(warehouse_event_dfs)

    def uppercase_all_columns(df: snowflake.snowpark.dataframe) -> snowflake.snowpark.dataframe:
        return df.select([F.col(column).as_(column.upper()) for column in df.columns])

    final_df.timestamp = final_df.timestamp.map(str)
    return final_df
#     df = session.create_dataframe(final_df)
#     uppercase_all_columns(df).write.mode("overwrite").save_as_table("wh_events_sim_python")


fast_simulation_df = smart_suspend(query_eras, 1)
fast_simulation_df

In [ ]:
import duckdb

ts = pd.date_range(qdf.era_start.min(), qdf.era_end.max() + pd.Timedelta(minutes=1.5), freq='1S')
df_time = pd.DataFrame(ts, columns=['ts'])
df_time

df = duckdb.query("""
with timeseries as (
    select * from df_time
)
select
    ts,
    count(era_start) > 0 as active
from 
    timeseries t
left join qdf 
on t.ts between qdf.era_start and qdf.era_end
group by 1
order by ts
""").df()

df

import plotly.express as px
fig = px.scatter(
    df,
    x='ts',
    y='active'
)
fig.update_yaxes(autorange="reversed")

fig.show()

In [ ]:


# state = list(zip(df.timestamp, df.eras_found))

# def wh_events_simulated(wh_activity_series):

state = list(zip(df.ts, df.active))
started = False
events = []

def maybe_suspend(started, active, time_since_resumed):
    # smart shutdown
    return started and (not active) and (time_since_resumed >= Timedelta(seconds=60))

for (ts, active) in state:
    # print(f"{ts}, {active}")
    if active and not started:
        started = True
        resumed_on = ts
        events.append((ts, 'resume'))

    if maybe_suspend(started, active, ts - resumed_on):
        started = False
        events.append((ts, 'suspend'))

test_events_df = pd.DataFrame(events, columns=['timestamp', 'event_type'])
test_events_df

In [ ]:
expected_events = [
 {'timestamp': Timestamp('2023-04-10 16:44:00'), 'event_type': 'resume'},
 {'timestamp': Timestamp('2023-04-10 16:45:06'), 'event_type': 'suspend'},
 {'timestamp': Timestamp('2023-04-10 16:45:15'), 'event_type': 'resume'},
 {'timestamp': Timestamp('2023-04-10 16:46:15'), 'event_type': 'suspend'}
]

expected_df = pd.DataFrame.from_dict(expected_events)

pd.testing.assert_frame_equal(test_events_df, expected_df)
pd.testing.assert_frame_equal(fast_simulation_df, expected_df)

In [ ]:
df = sql_to_df("show warehouses")
df

In [ ]:
managed = {
       "account": [
            'DEMO'
       ]
}

def maybe_suspend_warehouses(whs):
    logger.info("======== processing warehouses ===============================================================================================\n")
    df = sql_to_df("show warehouses")
    states = df.set_index('name', drop=False).to_dict('index')
    
    for wh in whs:
        logger.info(f"======== processing warehouse {wh} ===============================================================================================\n")
        status = states[wh]
        started = status['state'] == 'STARTED'
        resumed_on = status['resumed_on']
        time_since_resumed = pd.Timestamp.utcnow() - resumed_on
        active = (status['running'] + status['queued']) > 0
        
        logger.info(f"started {started} | active {active} | time_since_resumed {time_since_resumed}")

        suspend = maybe_suspend(started, active, time_since_resumed) 
        if suspend:
            wh = status['name']
            logger.info(f"SUSPENDING WAREHOUSE {wh}")
            # run_sql(f"ALTER WAREHOUSE {wh} SUSPEND")



In [ ]:
import threading

print("Number of active threads:", threading.active_count())

In [ ]:
while True:
    time.sleep(1)
    maybe_suspend_warehouses(managed)

In [ ]:
import duckdb
duckdb.query("select * from wh_df")

In [ ]:
duckdb.query("""
select  * from
generate_series(
        (select min(era_start) from wh_df),
        (select max(era_end) from wh_df),
        interval 1 second
    );
""")

In [ ]:
duckdb.query("""
SELECT 
* 
FROM 
generate_series(
    (select min(era_start) from wh_df)::timestamp,
    TIMESTAMP '2021-11-29', 
    INTERVAL 1 DAY
);

""")

In [ ]:
duckdb.query("""
SELECT * FROM generate_series(TIMESTAMP '2021-01-01', TIMESTAMP '2021-11-29', INTERVAL 1 DAY);

""")